In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import torch
import torch.nn.functional as F
from xformers.ops.fmha import (
    memory_efficient_attention_forward,
    memory_efficient_attention_backward, 
    memory_efficient_attention_partial,
    merge_attentions
)
import torch

In [3]:
head_num = 16
dim = 128
seq_len = 200
chunk_size = 4
batch_size = 1

In [4]:
q = torch.randn(batch_size, head_num, seq_len, dim).cuda().to(torch.bfloat16)
k = torch.randn(batch_size, head_num, seq_len, dim).cuda().to(torch.bfloat16)
v = torch.randn(batch_size, head_num, seq_len, dim).cuda().to(torch.bfloat16)

In [5]:
out_dot = torch.nn.functional.scaled_dot_product_attention(q, k, v, )

In [6]:
q_ = q.transpose(1, 2)
k_ = k.transpose(1, 2)
v_ = v.transpose(1, 2)
v_.shape

torch.Size([1, 200, 16, 128])

In [8]:
Q_block = q_chunks[0]

In [13]:
def local_attention(Q_block, k_chunks, v_chunks):
    outs, max_lse = None, None
    new_denominator = None
    attn_output = None
    new_lse_full = None

    for i in range(len(k_chunks)):
        out_, lse_ = memory_efficient_attention_partial(Q_block, k_chunks[i], v_chunks[i])
        lse_ = lse_.transpose(1, 2)

        if i == 0:
            max_lse = lse_
            adjust_factors = torch.ones_like(lse_).unsqueeze(-1)
            new_denominator = adjust_factors
            attn_output = out_ * adjust_factors
            new_lse_full = lse_
        else:
            new_max_lse = torch.maximum(max_lse, lse_)

            old_adjust_factors = torch.exp(max_lse - new_max_lse).unsqueeze(-1)
            new_adjust_factors = torch.exp(lse_ - new_max_lse).unsqueeze(-1)

            new_denominator = old_adjust_factors * new_denominator + new_adjust_factors
            attn_output = old_adjust_factors * attn_output + new_adjust_factors * out_
            new_lse_full = new_max_lse + torch.log(torch.exp(new_lse_full - new_max_lse) + torch.exp(lse_ - new_max_lse))

            max_lse = new_max_lse
    
    return attn_output, max_lse, new_lse_full

In [14]:
# first node
attn_output_0, max_lse_0, new_lse_full_0 = local_attention(Q_block, k_chunks[:2], v_chunks[:2])
# second node
attn_output_1, max_lse_1, new_lse_full_1 = local_attention(Q_block, k_chunks[2:], v_chunks[2:])

In [15]:
attn_outputs = [attn_output_0, attn_output_1]
max_lses = [max_lse_0, max_lse_1]

In [16]:
denominators = []
adjusted_outputs = []

all_lses = torch.cat([lse.unsqueeze(0) for lse in max_lses], dim=0)
global_max_lse = torch.max(all_lses, dim=0)[0]

for i, (output, lse) in enumerate(zip(attn_outputs, max_lses)):
    adjust_factor = torch.exp(lse - global_max_lse).unsqueeze(-1)

    adjusted_outputs.append(output * adjust_factor)
    denominators.append(adjust_factor)

final_output = torch.zeros_like(adjusted_outputs[0])
final_denominator = torch.zeros_like(denominators[0])

for adj_output, denom in zip(adjusted_outputs, denominators):
    final_output += adj_output
    final_denominator += denom

merged_output = (final_output / final_denominator).transpose(1, 2)

In [17]:
(merged_output.argmax(-1) == out_dot[:,:,:seq_len // chunk_size].argmax(-1)).float().mean()

tensor(0.9962, device='cuda:0')

In [18]:
(merged_output.sign() == out_dot[:,:,:seq_len // chunk_size].sign()).float().mean()

tensor(0.9993, device='cuda:0')

In [ ]:
# aggregate!